In [ ]:
import requests
from requests import ConnectionError,HTTPError,Timeout
import numpy as np
import pandas as pd

url = 'https://data.ntpc.gov.tw/api/datasets/010e5b15-3823-4b20-b401-b1cf000550c5/json?page=0&size=1000'
try:
    response = requests.get(url)    
    response.raise_for_status()    
except ConnectionError:
    print('找不到伺服器')
except HTTPError:
    print('網頁找不到')
except Timeout:
    print('超過時間沒有回應')
else:
    print('沒有發生問題')

records = response.json()
#records
allRecords = pd.DataFrame(records,columns=['sna','tot','mday','sbi','sarea','ar','bemp'])
allRecords1 = allRecords.rename(columns= {'sna':'站名', 'sarea':'區域', 'ar':'地址', 'tot':'數量','sbi':'可借', 'bemp':'可還','mday':'時間'})

沒有發生問題


,站名,數量,時間,可借,區域,地址,可還
0,YouBike2.0_下庄市場,20,20250615150803,14,八里區,舊城路21號(前),6
1,YouBike2.0_八里行政中心,20,20250615154903,6,八里區,十三行路(靠中山路二段路口西南側人行道),14
2,YouBike2.0_八里中庄市場綜合大樓,28,20250615152804,3,八里區,中山路一段268巷2號(對面汽車停車場),25
3,YouBike2.0_大崁國小,20,20250615153704,3,八里區,忠八街2號(前),17
4,YouBike2.0_龍形停車場,40,20250615155703,3,八里區,龍米路一段318號(對面停車場),37
...,...,...,...,...,...,...,...
995,YouBike2.0_深坑松柏街,34,20250615111504,2,深坑區,松柏街24號西側,32
996,YouBike2.0_深坑區公所(深坑老街),30,20250615151403,8,深坑區,文化街45號西側,22
997,YouBike2.0_東南科技大學,18,20250615155517,0,深坑區,北深路三段170號南側,17
998,YouBike2.0_深坑雲鄉路,12,20250615135802,3,深坑區,雲鄉路1號北側,9
